**Ejercicio 2:** Implemente el método de clustering k-medias sobre el conjunto de datos Iris (GTP2) y compare las soluciones obtenidas con las de un SOM en estos mismos datos con una matriz de contingencia.

#### <b>Librerías</b>

In [1]:
import random
import csv
import numpy as np

from som_func import som

from sklearn.metrics.cluster import contingency_matrix

#### **Inicialización**

Para este método, el único parámetro que debemos cambiar es *k*, la cantidad de centroides o de grupos que queremos formar. También el nombre del archivo del cual levantaremos los datos de entrada, en este caso, el conjunto de datos *Iris* de la guía 2. Para poder comparar la salida de k-medias y SOM con el **golden standard**, también guardamos en un vector las salidas esperadas que nos proporciona el .csv.

In [2]:
k = 3           # Cantidad de centroides (y grupos)
cant_e = 4      # Cantidad de entradas en el archivo .csv

trn = np.loadtxt('./data/irisbin_trn.csv',delimiter=',')

inputs = np.empty(len(trn),dtype=object)      # Vector de entradas
yd = np.empty(len(trn),dtype=object)          # Salida esperadas

for i in range(len(trn)):
    fila = trn[i]
    yd[i] = fila[cant_e:]
    inputs[i] = fila[0:cant_e]

Teniendo la cantidad de centroides deseada, inicializamos al azar las coordenadas de los mismos. Existen varias formas de hacer esto:

1. Inicializo al azar las coordenadas *x_j* para cada centroide *c_j*.
2. Tomo patrones al azar y les asigno sus coordenadas a los centroides.
3. Inicializo al azar los grupos correspondientes a cada patrón.

In [3]:
# Con la inicialización 2:
c = np.empty(k,dtype=object)
for i in range(k):
    c[i] = random.choice(inputs)

#### **Adaptación de centroides**

Trabajo por épocas como en otros métodos. Primero debo asignarle el centroide más cercano a cada patrón de entradas. Para esto, mido las distancias y guardo el índice del centroide más cercano en un vector del tamaño del vector *inputs* que almacena los patrones como filas.

Una vez asignados los centroides a todos los patrones de entrada, puedo recalcular las coordenadas de los centroides para que queden en el medio de los patrones que abarcan.

En la siguiente época voy a reasignar los centroides para cada patrón y corto el algoritmo cuando deje de reasignar patrones (convirgió a una solución).

In [4]:
max_epoca = 50
epoca = 1
last_input_centroid = np.empty(len(inputs))     # Vector donde guardo la última asignación de centroides a patrones
input_centroid = np.empty(len(inputs))          # Vector donde guardo la asignación actual de centroides a patrones

while epoca <= max_epoca:
    flag = False
    
    for i in range(len(inputs)):                # Asigno centroides a patrones:
        dist = [np.linalg.norm(inputs[i] - c[j]) for j in range(k)]
        if (np.argmin(dist) != input_centroid[i]):
            input_centroid[i] = np.argmin(dist)
            flag = True     # Si reasigné, pongo la bandera en true.

    for j in range(k):                          # Recalculo coordenadas de centroides
        index_inputs_cj = np.where(input_centroid == j)[0]
        if(len(index_inputs_cj) != 0):          # Evito dividir por cero en el promedio
            suma = sum(inputs[i] for i in index_inputs_cj)
            c[j] = suma/len(index_inputs_cj)

    #print(np.histogram(input_centroid,range(k+1)))     # Distribución por época
    if (flag == False):  # Si no reasigné nunca, corto.
        break

    epoca += 1

print('Finalizó en la época',epoca,'con la siguiente distribución de clases:')
print(np.histogram(input_centroid,range(k+1))[0])
print('Las salidas deseadas tienen la distribución:')
idx_yd = [np.argmax(v) for v in yd]
print(np.histogram(idx_yd,range(k+1))[0])

Finalizó en la época 5 con la siguiente distribución de clases:
[46 20 45]
Las salidas deseadas tienen la distribución:
[34 32 45]


#### **Gráficas y conclusiones:**

Analizamos los resultados con una matriz de contingencia. Comparamos K-medias contra Golden Standard, K-medias contra SOM y SOM contra Golden Standard para determinar la eficiencia de los métodos.

En los resultados vamos a ver que no siempre aparecen los True Positives en la diagonal principal porque cuando separa en grupos lo hace con índices distintos a los de la codificación del Iris, pero vemos que hay sólo un elemento por fila de la matriz que tiene un número alto (con ciertos errores).

Comenzamos calculando el SOM con una función auxiliar del ejercicio anterior.

In [5]:
filas = 5
columnas = 5
rv = 2
cant_e = 4
archivo = './data/irisbin_trn.csv'

inputs_som,input_neurona = som(filas,columnas,rv,cant_e,archivo)

**K-medias contra Golden Standard**

In [6]:
matrix = contingency_matrix(idx_yd,input_centroid)
print('Matriz de contingencia:\n',matrix)

Matriz de contingencia:
 [[14 20  0]
 [32  0  0]
 [ 0  0 45]]


**K-medias contra SOM**

In [7]:
matrix = contingency_matrix(input_centroid,input_neurona)
print('Matriz de contigencia:\n',matrix)

Matriz de contigencia:
 [[ 1  0  0 37  8]
 [ 0  0 20  0  0]
 [ 0 45  0  0  0]]


**SOM contra Golden Standard**

In [8]:
matrix = contingency_matrix(idx_yd,input_neurona)
print('Matriz de contigencia:\n',matrix)

Matriz de contigencia:
 [[ 0  0 20 14  0]
 [ 1  0  0 23  8]
 [ 0 45  0  0  0]]
